In [1]:
from datetime import datetime
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
from PIL import Image
from typing import Tuple, Optional
from torchvision import models

import os
import sys
import json
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import random

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set random seed for reproducibility
torch.manual_seed(789)
np.random.seed(789)
random.seed(789)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(789)

Using device: cuda


# 1. Model and Trainer

## 1.1 Model Definition

In [2]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torchvision.models as models

class InceptionV3Encoder(nn.Module):
    """InceptionV3 encoder as used in the original Nutrition5k paper"""
    
    def __init__(self, pretrained: bool = False, in_channels: int = 3):
        super().__init__()
        
        # Load InceptionV3 model
        inception = models.inception_v3(pretrained=pretrained, aux_logits=False)
        
        # The output of InceptionV3 features is 2048 channels
        self.out_channels = 2048
        
        # Modify first conv if we have different input channels (e.g., 1 for depth)
        if in_channels != 3:
            self.Conv2d_1a_3x3 = nn.Conv2d(
                in_channels, 32, kernel_size=3, stride=2, bias=False
            )
        else:
            self.Conv2d_1a_3x3 = inception.Conv2d_1a_3x3
        
        # Copy all other layers from InceptionV3
        # First block
        self.Conv2d_2a_3x3 = inception.Conv2d_2a_3x3
        self.Conv2d_2b_3x3 = inception.Conv2d_2b_3x3
        self.maxpool1 = inception.maxpool1
        
        # Second block
        self.Conv2d_3b_1x1 = inception.Conv2d_3b_1x1
        self.Conv2d_4a_3x3 = inception.Conv2d_4a_3x3
        self.maxpool2 = inception.maxpool2
        
        # Inception blocks
        self.Mixed_5b = inception.Mixed_5b
        self.Mixed_5c = inception.Mixed_5c
        self.Mixed_5d = inception.Mixed_5d
        self.Mixed_6a = inception.Mixed_6a
        self.Mixed_6b = inception.Mixed_6b
        self.Mixed_6c = inception.Mixed_6c
        self.Mixed_6d = inception.Mixed_6d
        self.Mixed_6e = inception.Mixed_6e
        self.Mixed_7a = inception.Mixed_7a
        self.Mixed_7b = inception.Mixed_7b
        self.Mixed_7c = inception.Mixed_7c
    
    def forward(self, x):
        """
        Args:
            x: Input tensor (B, C, H, W)
        Returns:
            Feature map (B, 2048, H/32, W/32)
        """
        # First block
        x = self.Conv2d_1a_3x3(x)
        x = self.Conv2d_2a_3x3(x)
        x = self.Conv2d_2b_3x3(x)
        x = self.maxpool1(x)
        
        # Second block
        x = self.Conv2d_3b_1x1(x)
        x = self.Conv2d_4a_3x3(x)
        x = self.maxpool2(x)
        
        # Inception blocks
        x = self.Mixed_5b(x)
        x = self.Mixed_5c(x)
        x = self.Mixed_5d(x)
        x = self.Mixed_6a(x)
        x = self.Mixed_6b(x)
        x = self.Mixed_6c(x)
        x = self.Mixed_6d(x)
        x = self.Mixed_6e(x)
        x = self.Mixed_7a(x)
        x = self.Mixed_7b(x)
        x = self.Mixed_7c(x)
        
        return x

# Early Fusion Module (RGB + Depth fused at input level)
class EarlyFusion(nn.Module):
    """
    Early Fusion: Combine RGB and Depth channels at the input level
    before processing through the network
    """
    
    def __init__(self, pretrained: bool = False, fusion_channels: int = 2048, dropout_rate: float = 0.4):
        super().__init__()
        
        # Create a single encoder with 4 input channels (3 RGB + 1 Depth)
        self.encoder = InceptionV3Encoder(pretrained=pretrained, in_channels=4)
        
        # Regression head for calorie prediction
        self.regression_head = RegressionHead(
            in_channels=self.encoder.out_channels,
            dropout_rate=dropout_rate
        )
    
    def forward(self, rgb, depth):
        """
        Args:
            rgb: RGB images (B, 3, H, W)
            depth: Depth images (B, 1, H, W)
        
        Returns:
            Predicted calories (B, 1)
        """
        # Concatenate RGB and depth along channel dimension
        x = torch.cat([rgb, depth], dim=1)  # (B, 4, H, W)
        
        # Process through the encoder
        features = self.encoder(x)
        
        # Predict calories
        calories = self.regression_head(features)
        
        return calories

# Late Fusion Module (RGB + Depth processed separately and fused at regression level)
class LateFusion(nn.Module):
    """
    Late Fusion: Process RGB and Depth streams independently, then fuse at the regression head level
    """
    
    def __init__(self, pretrained: bool = False, fusion_channels: int = 2048, dropout_rate: float = 0.4):
        super().__init__()
        
        # RGB and Depth encoders
        self.rgb_encoder = InceptionV3Encoder(pretrained=pretrained, in_channels=3)
        self.depth_encoder = InceptionV3Encoder(pretrained=pretrained, in_channels=1)
        
        # Global average pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Fusion at the feature vector level
        in_features = self.rgb_encoder.out_channels + self.depth_encoder.out_channels
        
        # Fully connected layers for regression
        self.regression_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 1)
        )
    
    def forward(self, rgb, depth):
        """
        Args:
            rgb: RGB images (B, 3, H, W)
            depth: Depth images (B, 1, H, W)
        
        Returns:
            Predicted calories (B, 1)
        """
        # Extract features from both streams
        rgb_features = self.rgb_encoder(rgb)    # (B, 2048, H/32, W/32)
        depth_features = self.depth_encoder(depth)  # (B, 2048, H/32, W/32)
        
        # Apply global average pooling
        rgb_features = self.avgpool(rgb_features)    # (B, 2048, 1, 1)
        depth_features = self.avgpool(depth_features)  # (B, 2048, 1, 1)
        
        # Concatenate feature vectors
        fused = torch.cat([rgb_features, depth_features], dim=1)  # (B, 4096, 1, 1)
        
        # Predict calories
        calories = self.regression_layers(fused)
        
        return calories

class RegressionHead(nn.Module):
    def __init__(self, in_channels: int = 2048, dropout_rate: float = 0.4):
        super().__init__()
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_channels, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        x = self.avgpool(x)  # (B, C, 1, 1)
        x = self.fc_layers(x)  # (B, 1)
        return x


class VolumeEstimator(nn.Module):
    """
    Food volume estimation from overhead depth images following the Nutrition5k paper.
    
    Given:
    - Distance between camera and capture plane: 35.9 cm
    - Per-pixel surface area at this distance: 5.957 × 10^-3 cm²
    
    The volume is calculated by:
    1. Computing per-pixel volume (depth × surface_area)
    2. Summing over all food pixels (using binary threshold segmentation)
    """
    
    def __init__(self, 
                 camera_distance: float = 35.9,  # cm
                 pixel_surface_area: float = 5.957e-3,  # cm²
                 depth_threshold: float = 0.1):  # Threshold for simple segmentation
        super().__init__()
        
        self.camera_distance = camera_distance
        self.pixel_surface_area = pixel_surface_area
        self.depth_threshold = depth_threshold
    
    def forward(self, depth_images):
        """
        Args:
            depth_images: Depth images (B, 1, H, W), normalized to [0, 1] range
        
        Returns:
            volume_estimates: Volume in cm³ for each image (B, 1)
        """
        # Simple threshold-based segmentation for foreground/background
        segmentation_mask = (depth_images > self.depth_threshold).float()
        
        # Convert normalized depth back to actual depth values
        # Assuming depth is normalized to [0, 1] and represents distance from camera
        # For simplicity, we assume the depth represents actual distance in cm scaled to [0, 1]
        depth_cm = depth_images * self.camera_distance
        
        # Calculate per-pixel volume: depth × surface_area
        per_pixel_volume = depth_cm * self.pixel_surface_area  # (B, 1, H, W)
        
        # Apply segmentation mask to consider only food pixels
        masked_volume = per_pixel_volume * segmentation_mask
        
        # Sum over all pixels to get total volume
        volume_estimates = masked_volume.sum(dim=[2, 3])  # (B, 1)
        
        return volume_estimates


class RegressionHeadWithVolume(nn.Module):
    """
    Regression head that concatenates volume estimate to InceptionV3 features.
    
    According to the paper: "concatenating the volume estimation value to the output 
    of the InceptionV3 backbone, before the following two fully connected layers"
    with FC layers of 64 and 1 dimension.
    """
    
    def __init__(self, in_channels: int = 2048, dropout_rate: float = 0.4):
        super().__init__()
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Two FC layers as described in the paper (2048+1 -> 64 -> 1)
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_channels + 1, 64),  # +1 for volume
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            nn.Linear(64, 1)
        )
    
    def forward(self, features, volume):
        """
        Args:
            features: Feature maps from backbone (B, 2048, H, W)
            volume: Volume estimates (B, 1)
        
        Returns:
            Predicted calories (B, 1)
        """
        # Global average pooling
        x = self.avgpool(features)  # (B, 2048, 1, 1)
        x = torch.flatten(x, 1)  # (B, 2048)
        
        # Concatenate volume estimate
        x = torch.cat([x, volume], dim=1)  # (B, 2049)
        
        # Predict calories
        x = self.fc_layers(x)  # (B, 1)
        
        return x

class Nutrition5kModel(nn.Module):
    """
    Implementation of the dual-stream architecture used in the original Nutrition5k paper
    Uses InceptionV3 as the backbone and middle fusion
    """
    
    def __init__(
        self,
        fusion: str = 'middle',
        fusion_channels: int = 2048,
        dropout_rate: float = 0.4,
        pretrained: bool = False,
        use_volume: bool = False
    ):
        super().__init__()
        
        self.use_volume = use_volume
        
        if fusion == 'early':
            self.model = EarlyFusion(
                pretrained=pretrained,
                fusion_channels=fusion_channels,
                dropout_rate=dropout_rate
            )
        elif fusion == 'late':
            self.model = LateFusion(
                pretrained=pretrained,
                fusion_channels=fusion_channels,
                dropout_rate=dropout_rate
            )
        elif fusion == 'image_only':
            # Image-only variant: only RGB is used
            self.rgb_encoder = InceptionV3Encoder(pretrained=pretrained, in_channels=3)
            
            # Volume estimator (if enabled)
            if use_volume:
                self.volume_estimator = VolumeEstimator()
                self.regression_head = RegressionHeadWithVolume(
                    in_channels=self.rgb_encoder.out_channels,
                    dropout_rate=dropout_rate
                )
            else:
                self.regression_head = RegressionHead(
                    in_channels=self.rgb_encoder.out_channels,
                    dropout_rate=dropout_rate
                )
        elif fusion == 'image_volume':
            # Image+Volume variant: RGB encoder + volume as additional signal
            self.rgb_encoder = InceptionV3Encoder(pretrained=pretrained, in_channels=3)
            self.volume_estimator = VolumeEstimator()
            self.regression_head = RegressionHeadWithVolume(
                in_channels=self.rgb_encoder.out_channels,
                dropout_rate=dropout_rate
            )
            self.use_volume = True  # Always use volume for this variant
        else:  # middle fusion
            # RGB and Depth encoders using InceptionV3
            self.rgb_encoder = InceptionV3Encoder(pretrained=pretrained, in_channels=3)
            self.depth_encoder = InceptionV3Encoder(pretrained=pretrained, in_channels=1)
            
            # Create middle fusion module
            from_channels = self.rgb_encoder.out_channels + self.depth_encoder.out_channels
            self.fusion_conv = nn.Sequential(
                nn.Conv2d(from_channels, fusion_channels, kernel_size=1, bias=False),
                nn.BatchNorm2d(fusion_channels),
                nn.ReLU(inplace=True)
            )
            
            # Volume estimator (if enabled)
            if use_volume:
                self.volume_estimator = VolumeEstimator()
                self.regression_head = RegressionHeadWithVolume(
                    in_channels=fusion_channels,
                    dropout_rate=dropout_rate
                )
            else:
                self.regression_head = RegressionHead(
                    in_channels=fusion_channels,
                    dropout_rate=dropout_rate
                )
    
    def forward(self, rgb, depth):
        """
        Args:
            rgb: RGB images (B, 3, H, W)
            depth: Depth images (B, 1, H, W)
        
        Returns:
            calorie_pred: Predicted calories (B, 1)
        """
        if hasattr(self, 'model'):
            return self.model(rgb, depth)
        
        # Calculate volume estimate if enabled
        volume = None
        if self.use_volume and hasattr(self, 'volume_estimator'):
            volume = self.volume_estimator(depth)  # (B, 1)
        
        # Image-only or Image+Volume variant
        if hasattr(self, 'rgb_encoder') and not hasattr(self, 'depth_encoder'):
            rgb_features = self.rgb_encoder(rgb)  # (B, 2048, H/32, W/32)
            
            if volume is not None:
                calorie_pred = self.regression_head(rgb_features, volume)
            else:
                calorie_pred = self.regression_head(rgb_features)
            
            return calorie_pred
        
        # Extract features from both streams
        rgb_features = self.rgb_encoder(rgb)      # (B, 2048, H/32, W/32)
        depth_features = self.depth_encoder(depth)  # (B, 2048, H/32, W/32)
        
        # Middle fusion - concatenate and apply 1x1 conv
        fused = torch.cat([rgb_features, depth_features], dim=1)  # (B, 4096, H/32, W/32)
        fused = self.fusion_conv(fused)  # (B, 2048, H/32, W/32)
        
        # Predict calories (with or without volume)
        if volume is not None:
            calorie_pred = self.regression_head(fused, volume)
        else:
            calorie_pred = self.regression_head(fused)
        
        return calorie_pred
    
    def get_num_parameters(self):
        """Get total number of trainable parameters"""
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

# Factory function to build Nutrition5k models with different fusion types
def build_nutrition5k_model(fusion='middle', pretrained=False, dropout_rate=0.4, fusion_channels=2048, 
                           use_volume=False, **kwargs):
    """
    Factory function to build models using the Nutrition5k paper architecture (InceptionV3 backbone)
    
    Args:
        fusion: Fusion type ('early', 'middle', 'late', 'image_only', or 'image_volume')
        pretrained: Whether to use pretrained weights for InceptionV3
        dropout_rate: Dropout rate for regression head
        fusion_channels: Number of channels after fusion
        use_volume: Whether to use volume estimation as additional signal (uses simple threshold-based segmentation)
    
    Returns:
        Nutrition5k model with specified configuration
    """
    return Nutrition5kModel(
        fusion=fusion,
        fusion_channels=fusion_channels,
        dropout_rate=dropout_rate,
        pretrained=pretrained,
        use_volume=use_volume
    )

## 1.2 Trainer Definition

In [3]:
import math

def get_warmup_cosine_scheduler(optimizer, warmup_steps, total_steps, min_lr_ratio=0.0):
    def lr_lambda(current_step):
        if current_step < warmup_steps:
            return float(current_step) / float(max(1, warmup_steps))
        else:
            progress = float(current_step - warmup_steps) / float(max(1, total_steps - warmup_steps))
            return min_lr_ratio + (1.0 - min_lr_ratio) * 0.5 * (1.0 + math.cos(math.pi * progress))
    
    return optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)


class EarlyStopping:
    """Early stopping to stop training when validation loss stops improving"""
    
    def __init__(self, patience: int = 10, min_delta: float = 0.0, mode: str = 'min'):
        """
        Args:
            patience: Number of epochs with no improvement after which training will be stopped
            min_delta: Minimum change to qualify as an improvement
            mode: 'min' or 'max' - whether lower or higher metric is better
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_epoch = 0
        
    def __call__(self, score, epoch):
        if self.best_score is None:
            self.best_score = score
            self.best_epoch = epoch
            return False
        
        if self.mode == 'min':
            improved = score < (self.best_score - self.min_delta)
        else:
            improved = score > (self.best_score + self.min_delta)
        
        if improved:
            self.best_score = score
            self.best_epoch = epoch
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
                
        return self.early_stop


class Trainer:
    """Training manager for calorie prediction"""
    
    def __init__(
        self,
        model,
        train_loader,
        val_loader,
        criterion,
        optimizer,
        scheduler,
        device,
        output_dir,
        early_stopping_patience=15,
        scheduler_step_on_batch=False
    ):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.output_dir = output_dir
        self.scheduler_step_on_batch = scheduler_step_on_batch
        
        # Early stopping
        self.early_stopping = EarlyStopping(
            patience=early_stopping_patience,
            min_delta=0.1,
            mode='min'
        )
        
        # Tensorboard
        self.writer = SummaryWriter(log_dir=os.path.join(output_dir, 'tensorboard'))
        
        # Tracking
        self.best_val_loss = float('inf')
        self.train_losses = []
        self.val_losses = []
        self.best_metrics = {}
    
    def train_epoch(self):
        """Train for one epoch"""
        self.model.train()
        total_loss = 0.0
        num_batches = 0
        
        pbar = tqdm(self.train_loader, desc="Training")
        for batch_idx, batch in enumerate(pbar):
            # Move to device
            rgb = batch['rgb'].to(self.device)
            depth = batch['depth'].to(self.device)
            calories = batch['calorie'].to(self.device)
            
            # Forward pass
            self.optimizer.zero_grad()
            calorie_pred = self.model(rgb, depth)
            
            # Compute loss (MSE for calorie prediction)
            loss = self.criterion(calorie_pred.squeeze(), calories)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), max_norm=1.0)
            self.optimizer.step()
            
            # Update learning rate (if step_on_batch)
            if self.scheduler_step_on_batch and self.scheduler:
                self.scheduler.step()
            
            # Track metrics
            total_loss += loss.item()
            num_batches += 1
            
            # Update progress bar
            pbar.set_postfix({'Loss': f'{loss.item():.4f}'})
        
        return total_loss / num_batches
    
    def validate_epoch(self):
        """Validate for one epoch"""
        self.model.eval()
        total_loss = 0.0
        all_predictions = []
        all_targets = []
        
        with torch.no_grad():
            for batch in tqdm(self.val_loader, desc="Validation"):
                # Move to device
                rgb = batch['rgb'].to(self.device)
                depth = batch['depth'].to(self.device)
                calories = batch['calorie'].to(self.device)
                
                # Forward pass
                calorie_pred = self.model(rgb, depth)
                
                # Compute loss
                loss = self.criterion(calorie_pred.squeeze(), calories)
                total_loss += loss.item()
                
                # Store predictions and targets for metrics
                all_predictions.extend(calorie_pred.squeeze().cpu().numpy())
                all_targets.extend(calories.cpu().numpy())
        
        # Calculate metrics
        avg_loss = total_loss / len(self.val_loader)
        predictions = np.array(all_predictions)
        targets = np.array(all_targets)
        
        mae = np.mean(np.abs(predictions - targets))
        
        return avg_loss, mae
    
    def train(self, num_epochs):
        """Full training loop"""
        print(f"Starting training for {num_epochs} epochs...")
        
        for epoch in range(num_epochs):
            print(f"\nEpoch {epoch+1}/{num_epochs}")
            
            # Train
            train_loss = self.train_epoch()
            
            # Validate
            val_loss, mae = self.validate_epoch()
            
            # Update learning rate (if not step_on_batch)
            if not self.scheduler_step_on_batch and self.scheduler:
                self.scheduler.step(val_loss)
            
            # Log metrics
            self.writer.add_scalar('Loss/Train', train_loss, epoch)
            self.writer.add_scalar('Loss/Val', val_loss, epoch)
            self.writer.add_scalar('MAE', mae, epoch)
            
            # Save best model
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.best_metrics = {
                    'epoch': epoch + 1,
                    'val_loss': val_loss,
                    'mae': mae,
                }
                
                # Save model checkpoint
                torch.save({
                    'epoch': epoch + 1,
                    'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'val_loss': val_loss,
                    'mae': mae,
                }, os.path.join(self.output_dir, 'best_model.pth'))
            
            # Print epoch results
            print(f"Train Loss: {train_loss:.4f}")
            print(f"Val Loss: {val_loss:.4f}")
            print(f"MAE: {mae:.2f}")
            
            # Early stopping
            if self.early_stopping(val_loss, epoch):
                print(f"Early stopping triggered after {epoch+1} epochs")
                print(f"Best epoch: {self.early_stopping.best_epoch+1}")
                break
        
        self.writer.close()
        print(f"\nTraining completed!")
        print(f"Best validation loss: {self.best_val_loss:.4f}")


# 2. Dataset

## 2.1 Dataset Definition

In [ ]:
# Dataset Implementation
class Nutrition5KDataset(Dataset):
    """
    Dataset class for Nutrition5K with multi-modal inputs (RGB + Depth)
    """
    
    def __init__(
        self,
        csv_path: str,
        data_root: str,
        split: str = 'train',
        augment: bool = True,
        img_size: int = 224,
    ):
        self.data_root = data_root
        self.split = split
        self.augment = augment
        self.img_size = img_size
        
        # Load CSV
        self.df = pd.read_csv(csv_path)
        if 'Value' in self.df.columns and 'calories' not in self.df.columns:
            self.df = self.df.rename(columns={'Value': 'calories'})
        if 'calories' not in self.df.columns:
            raise ValueError("CSV file must contain a 'calories' column or a 'Value' column that can be renamed")
        self.df = self.df[self.df['calories'] < 3000].reset_index(drop=True)
                
        self.color_dir = os.path.join(data_root, 'color')
        self.depth_raw_dir = os.path.join(data_root, 'depth_raw')
        
        self.valid_indices = self._validate_dataset()
        print(f"Loaded {len(self.valid_indices)} valid samples out of {len(self.df)}")
        
        # Color normalization (ImageNet stats as baseline)
        self.color_normalize = T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
        
    def _validate_dataset(self):
        """This method ensure that the code don't break when there are corrupted images."""
        valid_indices = []
        
        for idx in range(len(self.df)):
            dish_id = self.df.iloc[idx]['ID']
            
            rgb_path = os.path.join(self.color_dir, dish_id, 'rgb.png')
            depth_path = os.path.join(self.depth_raw_dir, dish_id, 'depth_raw.png')
            
            # Check if files exist
            if not os.path.exists(rgb_path):
                continue
            if not os.path.exists(depth_path):
                continue
            
            # Try to load images to check for corruption
            try:
                with Image.open(rgb_path) as img:
                    img.verify()
                with Image.open(depth_path) as img:
                    img.verify()
                valid_indices.append(idx)
            except Exception as e:
                continue
                
        return valid_indices
    
    def __len__(self):
        return len(self.valid_indices)
    
    def _load_image_safe(self, path: str, mode: str = 'RGB') -> Optional[Image.Image]:
        """Safely load an image with error handling"""
        try:
            with Image.open(path) as img:
                return img.convert(mode).copy()
        except Exception as e:
            return None
    
    def _apply_augmentation(self, rgb_img, depth_img):
        """Apply geometric augmentation only (no color changes)"""
        if not self.augment:
            return rgb_img, depth_img
        
        # Convert to tensors first
        rgb_tensor = TF.to_tensor(rgb_img)
        depth_tensor = TF.to_tensor(depth_img)
        
        # Random horizontal flip
        if random.random() > 0.5:
            rgb_tensor = TF.hflip(rgb_tensor)
            depth_tensor = TF.hflip(depth_tensor)
        
        # Random rotation (±15 degrees)
        if random.random() > 0.5:
            angle = random.uniform(-15, 15)
            rgb_tensor = TF.rotate(rgb_tensor, angle)
            depth_tensor = TF.rotate(depth_tensor, angle)
        
        # Convert back to PIL
        rgb_img = TF.to_pil_image(rgb_tensor)
        depth_img = TF.to_pil_image(depth_tensor)
        
        return rgb_img, depth_img
    
    def _resize_and_center_crop(self, img, target_size: int = 256):
        """
        Resize and center crop image to target_size x target_size
        Matches the preprocessing in the Nutrition5k paper
        
        Args:
            img: PIL Image
            target_size: Target size (default 256x256 as per paper)
        
        Returns:
            Cropped PIL Image
        """
        # Get original dimensions
        width, height = img.size
        
        # Resize so the shorter side is target_size
        if width < height:
            new_width = target_size
            new_height = int(target_size * height / width)
        else:
            new_height = target_size
            new_width = int(target_size * width / height)
        
        img = img.resize((new_width, new_height), Image.LANCZOS)
        
        # Center crop to target_size x target_size
        left = (new_width - target_size) // 2
        top = (new_height - target_size) // 2
        right = left + target_size
        bottom = top + target_size
        
        img = img.crop((left, top, right, bottom))
        
        return img
    
    def __getitem__(self, idx):
        """Get a single sample"""
        actual_idx = self.valid_indices[idx]
        row = self.df.iloc[actual_idx]
        
        dish_id = row['ID']
        calorie = float(row['calories'])
        
        # Load images
        rgb_path = os.path.join(self.color_dir, dish_id, 'rgb.png')
        depth_path = os.path.join(self.depth_raw_dir, dish_id, 'depth_raw.png')
        
        rgb_img = self._load_image_safe(rgb_path, 'RGB')
        depth_img = self._load_image_safe(depth_path, 'L')  # Grayscale for depth
        
        # Fallback: return a black image
        if rgb_img is None or depth_img is None:
            rgb_img = Image.new('RGB', (self.img_size, self.img_size), (0, 0, 0))
            depth_img = Image.new('L', (self.img_size, self.img_size), 0)
        
        # Apply augmentation
        rgb_img, depth_img = self._apply_augmentation(rgb_img, depth_img)
        
        # Resize and center crop to match paper preprocessing (256x256)
        rgb_img = self._resize_and_center_crop(rgb_img, target_size=self.img_size)
        depth_img = self._resize_and_center_crop(depth_img, target_size=self.img_size)
        
        # Convert to tensors
        rgb_tensor = TF.to_tensor(rgb_img)  # (3, H, W)
        depth_tensor = TF.to_tensor(depth_img)  # (1, H, W)
        
        # Normalize RGB
        rgb_tensor = self.color_normalize(rgb_tensor)
        
        # Normalize depth (0-1 range, assuming depth is already in reasonable range)
        depth_tensor = depth_tensor / 255.0
        
        return {
            'dish_id': dish_id,
            'rgb': rgb_tensor,
            'depth': depth_tensor,
            'calorie': torch.tensor(calorie, dtype=torch.float32)
        }


def create_train_val_split(csv_path: str, val_ratio: float = 0.15, random_seed: int = 42):
    """
    Create train/validation split CSV files
    """
    # Read original CSV
    df = pd.read_csv(csv_path)    
    
    # Shuffle with fixed seed
    df_shuffled = df.sample(frac=1, random_state=random_seed).reset_index(drop=True)
    
    # Split
    val_size = int(len(df_shuffled) * val_ratio)
    train_df = df_shuffled[val_size:]
    val_df = df_shuffled[:val_size]
    
    # Save temporary CSV files
    base_dir = os.path.dirname(csv_path)
    train_csv = os.path.join(base_dir, 'train_split.csv')
    val_csv = os.path.join(base_dir, 'val_split.csv')
    
    train_df.to_csv(train_csv, index=False)
    val_df.to_csv(val_csv, index=False)
    
    return train_csv, val_csv

## 2.2 Dataset Loading

In [5]:
# Configuration - Update these paths to match your setup
DATA_ROOT = './Nutrition5K/Nutrition5K/train'  # Path to training data directory
CSV_PATH = './Nutrition5K/Nutrition5K/nutrition5k_train.csv'  # Path to training CSV
OUTPUT_DIR = './experiments'  # Directory to save experiment results

# Global training hyperparameters (learning rate and weight decay set per experiment)
BATCH_SIZE = 32
NUM_EPOCHS = 40
VAL_RATIO = 0.15
IMG_SIZE = 256
NUM_WORKERS = 4

print("Configuration:")
print(f"  Data root: {DATA_ROOT}")
print(f"  CSV path: {CSV_PATH}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Number of epochs: {NUM_EPOCHS}")
print(f"  Image size: {IMG_SIZE}")
print(f"  Workers: {NUM_WORKERS}")

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)


Configuration:
  Data root: ./Nutrition5K/Nutrition5K/train
  CSV path: ./Nutrition5K/Nutrition5K/nutrition5k_train.csv
  Output directory: ./experiments
  Batch size: 32
  Number of epochs: 40
  Image size: 256
  Workers: 4


### Dataset Loading

In [6]:
# Create train/validation split
print("Creating train/validation split...")
train_csv, val_csv = create_train_val_split(
    CSV_PATH,
    val_ratio=VAL_RATIO,
    random_seed=42
)

print(f"Train CSV: {train_csv}")
print(f"Validation CSV: {val_csv}")

# Load a sample to check data
sample_dataset = Nutrition5KDataset(
    csv_path=train_csv,
    data_root=DATA_ROOT,
    split='train',
    augment=False,  # No augmentation for checking
    img_size=IMG_SIZE,
)

print(f"\nDataset loaded successfully!")
print(f"Training samples: {len(sample_dataset)}")
print(f"RGB shape: {sample_dataset[0]['rgb'].shape}")
print(f"Depth shape: {sample_dataset[0]['depth'].shape}")


Creating train/validation split...
Train CSV: ./Nutrition5K/Nutrition5K/train_split.csv
Validation CSV: ./Nutrition5K/Nutrition5K/val_split.csv
Loaded 2804 valid samples out of 2805

Dataset loaded successfully!
Training samples: 2804
RGB shape: torch.Size([3, 256, 256])
Depth shape: torch.Size([1, 256, 256])


# 3. Experiments
We'll conduct experiments to compare different fusion strategies using the InceptionV3 architecture.

**Architecture**: InceptionV3
- **RGB encoder**: InceptionV3 (in_channels=3)
- **Depth encoder**: InceptionV3 (in_channels=1) 
- **Fusion**: Various fusion strategies (early, middle, late)
- **Volume estimation**: Optional food volume calculation from depth images

**Experiments**:
1. **InceptionV3 - Middle Fusion**: RGB and Depth features concatenated at feature map level, then fused with 1×1 conv
2. **InceptionV3 - Early Fusion**: RGB and Depth concatenated at input level (4 channels), processed by single encoder
3. **InceptionV3 - Late Fusion**: RGB and Depth processed separately, features concatenated after global pooling
4. **InceptionV3 - Image+Volume**: RGB encoder only + volume estimate from depth as additional signal
5. **InceptionV3 - Middle+Volume**: Middle fusion (RGB+Depth) + volume estimate as additional signal

## 3.1 InceptionV3 - Middle Fusion

In [ ]:
# Define experiment hyperparamers
# BATCH_SIZE = 32
NUM_EPOCHS = 40
DROPOUT_RATE = 0.4
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-6
EARLY_STOPPING_PATIENCE = 15
WARMUP_RATIO = 0.1
MIN_LR_RATIO = 0.05
FUSION_CHANNELS = 2048

def train_nutrition5k_model(fusion_type='middle'):
    """Train the Nutrition5k model with InceptionV3 and specified fusion type"""
    
    print("="*60)
    print(f"TRAINING: Nutrition5k InceptionV3 + {fusion_type.capitalize()} Fusion")
    print("="*60)
    
    # Create datasets
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Build model with specified fusion type
    model = build_nutrition5k_model(
        fusion=fusion_type,
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function
    criterion = nn.MSELoss()
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )
    
    print(f"Learning rate: {LEARNING_RATE}")
    print(f"Weight decay: {WEIGHT_DECAY}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)
    
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"inceptionv3_{fusion_type}_fusion_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, 'nutrition5k_experiments', exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Save experiment configuration
    config = {
        'fusion': fusion_type,
        'pretrained': False,
        'dropout_rate': DROPOUT_RATE,
        'fusion_channels': FUSION_CHANNELS,
        'learning_rate': LEARNING_RATE,
        'weight_decay': WEIGHT_DECAY,
        'batch_size': BATCH_SIZE,
        'img_size': IMG_SIZE,
        'num_epochs': NUM_EPOCHS
    }
    
    with open(os.path.join(exp_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics

# Run an experiment with middle fusion
middle_fusion_results = train_nutrition5k_model(fusion_type='middle')

TRAINING: Nutrition5k InceptionV3 + Middle Fusion
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 53,143,873
Training samples: 2804
Validation samples: 495
Learning rate: 0.0003
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.13it/s]


Train Loss: 99179.5700
Val Loss: 107412.9351
MAE: 240.64

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.31it/s]


Train Loss: 44613.2511
Val Loss: 22115.9072
MAE: 103.60

Epoch 3/40


...

Train Loss: 3230.0179
Val Loss: 7757.0370
MAE: 56.59

Epoch 39/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.55it/s]


Train Loss: 4668.2630
Val Loss: 12126.1959
MAE: 82.30

Epoch 40/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.40it/s]

Train Loss: 5625.5605
Val Loss: 14431.2035
MAE: 85.84

Training completed!
Best validation loss: 7578.5881

Experiment completed! Results saved to: ../experiments/nutrition5k_experiments/inceptionv3_middle_fusion_20251024_130923


## 3.2 InceptionV3 - Early Fusion

In [ ]:
# Define experiment hyperparamers
BATCH_SIZE = 32
NUM_EPOCHS = 40
DROPOUT_RATE = 0.4
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-6
EARLY_STOPPING_PATIENCE = 15
WARMUP_RATIO = 0.1
MIN_LR_RATIO = 0.05
FUSION_CHANNELS = 2048

def train_nutrition5k_model(fusion_type='middle'):
    """Train the Nutrition5k model with InceptionV3 and specified fusion type"""
    
    print("="*60)
    print(f"TRAINING: Nutrition5k InceptionV3 + {fusion_type.capitalize()} Fusion")
    print("="*60)
    
    # Create datasets
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Build model with specified fusion type
    model = build_nutrition5k_model(
        fusion=fusion_type,
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function
    criterion = nn.MSELoss()
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )
    
    print(f"Learning rate: {LEARNING_RATE}")
    print(f"Weight decay: {WEIGHT_DECAY}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)
    
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"inceptionv3_{fusion_type}_fusion_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, 'nutrition5k_experiments', exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Save experiment configuration
    config = {
        'fusion': fusion_type,
        'pretrained': False,
        'dropout_rate': DROPOUT_RATE,
        'fusion_channels': FUSION_CHANNELS,
        'learning_rate': LEARNING_RATE,
        'weight_decay': WEIGHT_DECAY,
        'batch_size': BATCH_SIZE,
        'img_size': IMG_SIZE,
        'num_epochs': NUM_EPOCHS
    }
    
    with open(os.path.join(exp_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics

# Run an experiment with middle fusion
early_fusion_results = train_nutrition5k_model(fusion_type='early')

TRAINING: Nutrition5k InceptionV3 + Early Fusion
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 22,966,465
Training samples: 2804
Validation samples: 495
Learning rate: 0.0003
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.54it/s]


Train Loss: 99325.6495
Val Loss: 107376.2798
MAE: 240.56

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.52it/s]


Train Loss: 42928.3323
Val Loss: 19980.8986
MAE: 103.48

Epoch 3/40


...

Train Loss: 2817.4882
Val Loss: 8004.2764
MAE: 54.92

Epoch 39/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.73it/s]


Train Loss: 3088.7458
Val Loss: 8371.1610
MAE: 58.62

Epoch 40/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.55it/s]

Train Loss: 2973.2887
Val Loss: 9626.2067
MAE: 60.45

Training completed!
Best validation loss: 7289.9086

Experiment completed! Results saved to: ../experiments/nutrition5k_experiments/inceptionv3_early_fusion_20251024_132025


## 3.3 InceptionV3 - Late Fusion

In [ ]:
# Define experiment hyperparamers
BATCH_SIZE = 32
NUM_EPOCHS = 40
DROPOUT_RATE = 0.4
LEARNING_RATE = 3e-4
WEIGHT_DECAY = 1e-6
EARLY_STOPPING_PATIENCE = 15
WARMUP_RATIO = 0.1
MIN_LR_RATIO = 0.05
FUSION_CHANNELS = 2048

def train_nutrition5k_model(fusion_type='middle'):
    """Train the Nutrition5k model with InceptionV3 and specified fusion type"""
    
    print("="*60)
    print(f"TRAINING: Nutrition5k InceptionV3 + {fusion_type.capitalize()} Fusion")
    print("="*60)
    
    # Create datasets
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Build model with specified fusion type
    model = build_nutrition5k_model(
        fusion=fusion_type,
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function
    criterion = nn.MSELoss()
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )
    
    print(f"Learning rate: {LEARNING_RATE}")
    print(f"Weight decay: {WEIGHT_DECAY}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)
    
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"inceptionv3_{fusion_type}_fusion_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, 'nutrition5k_experiments', exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Save experiment configuration
    config = {
        'fusion': fusion_type,
        'pretrained': False,
        'dropout_rate': DROPOUT_RATE,
        'fusion_channels': FUSION_CHANNELS,
        'learning_rate': LEARNING_RATE,
        'weight_decay': WEIGHT_DECAY,
        'batch_size': BATCH_SIZE,
        'img_size': IMG_SIZE,
        'num_epochs': NUM_EPOCHS
    }
    
    with open(os.path.join(exp_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics

# Run an experiment with middle fusion
late_fusion_results = train_nutrition5k_model(fusion_type='late')

TRAINING: Nutrition5k InceptionV3 + Late Fusion
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 45,799,745
Training samples: 2804
Validation samples: 495
Learning rate: 0.0003
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.52it/s]


Train Loss: 98389.8398
Val Loss: 107367.1013
MAE: 240.54

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.45it/s]


Train Loss: 41690.2808
Val Loss: 31997.4932
MAE: 123.44

Epoch 3/40


...

Train Loss: 4091.8209
Val Loss: 8399.1342
MAE: 57.81

Epoch 39/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.74it/s]


Train Loss: 4500.5079
Val Loss: 9125.9881
MAE: 62.44

Epoch 40/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.55it/s]

Train Loss: 3323.3737
Val Loss: 8567.3797
MAE: 60.29

Training completed!
Best validation loss: 7766.9089

Experiment completed! Results saved to: ../experiments/nutrition5k_experiments/inceptionv3_late_fusion_20251024_133101


## 3.4 InceptionV3 - Image + Volume

This section implements the food volume estimation method as described in the Nutrition5k paper. The method:

1. **Estimates food volume from overhead depth images** using:
   - Camera distance: 35.9 cm
   - Per-pixel surface area: 5.957 × 10⁻³ cm²
   
2. **Uses binary foreground/background segmentation** to identify food pixels

3. **Calculates volume** by summing per-pixel volumes (depth × surface_area) over all food pixels

4. **Concatenates volume estimate** to the InceptionV3 backbone output before FC layers

We implement three variants:
- **Image+Volume**: RGB + volume estimate as additional signal  
- **Middle+Volume**: RGB + Depth fusion + volume estimate


In [ ]:
# Define experiment hyperparamers
BATCH_SIZE = 32
NUM_EPOCHS = 40
DROPOUT_RATE = 0.4
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 1e-6
EARLY_STOPPING_PATIENCE = 15
WARMUP_RATIO = 0.1
MIN_LR_RATIO = 0.05
FUSION_CHANNELS = 2048

# Training function with volume estimation support
def train_nutrition5k_with_volume(fusion_type='image_volume'):
    """
    Train the Nutrition5k model with volume estimation
    
    Args:
        fusion_type: 'image_only', 'image_volume', 'middle', etc.
        use_segmentation: Whether to use learned segmentation for volume estimation
    """
    
    print("="*60)
    print(f"TRAINING: Nutrition5k InceptionV3 + {fusion_type.upper()}")

    print("="*60)
    
    # Create datasets
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    

   # Use Volume    
    model = build_nutrition5k_model(
        fusion=fusion_type,
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS,
        use_volume=True,
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function
    criterion = nn.MSELoss()
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )
    
    print(f"Learning rate: {LEARNING_RATE}")
    print(f"Weight decay: {WEIGHT_DECAY}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)
    
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"inceptionv3_{fusion_type}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, 'nutrition5k_experiments', exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Save experiment configuration
    config = {
        'fusion': fusion_type,
        'use_volume': True,
        'pretrained': False,
        'dropout_rate': DROPOUT_RATE,
        'fusion_channels': FUSION_CHANNELS,
        'learning_rate': LEARNING_RATE,
        'weight_decay': WEIGHT_DECAY,
        'batch_size': BATCH_SIZE,
        'img_size': IMG_SIZE,
        'num_epochs': NUM_EPOCHS
    }
    
    with open(os.path.join(exp_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics


image_volume_result = train_nutrition5k_with_volume(fusion_type='image_volume')

TRAINING: Nutrition5k InceptionV3 + IMAGE_VOLUME
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 21,916,833
Training samples: 2804
Validation samples: 495
Learning rate: 0.0005
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.60it/s]


Train Loss: 98651.2712
Val Loss: 107381.6948
MAE: 240.57

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.67it/s]


Train Loss: 65919.9727
Val Loss: 29319.6537
MAE: 118.82

Epoch 3/40


...

Validation: 100%|██████████| 16/16 [00:02<00:00,  6.61it/s]


Train Loss: 4822.6640
Val Loss: 7814.0743
MAE: 54.46

Epoch 40/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.48it/s]

Train Loss: 5745.1829
Val Loss: 9136.1897
MAE: 59.73

Training completed!
Best validation loss: 7175.3438

Experiment completed! Results saved to: ../experiments/nutrition5k_experiments/inceptionv3_image_volume_20251024_140149


: 

## 3.5 InceptionV3 - Image + Depth + Volume

In [9]:
# Configure experiment settings
BATCH_SIZE = 32
NUM_EPOCHS = 40
DROPOUT_RATE = 0.4
LEARNING_RATE = 5e-4
WEIGHT_DECAY = 1e-6
EARLY_STOPPING_PATIENCE = 15
WARMUP_RATIO = 0.1
MIN_LR_RATIO = 0.05
FUSION_CHANNELS = 2048  # InceptionV3 output channels

# Training function with volume estimation support
def train_nutrition5k_with_volume(fusion_type='image_volume'):
    """
    Train the Nutrition5k model with volume estimation
    
    Args:
        fusion_type: 'image_only', 'image_volume', 'middle', etc.
        use_segmentation: Whether to use learned segmentation for volume estimation
    """
    
    print("="*60)
    print(f"TRAINING: Nutrition5k InceptionV3 + {fusion_type.upper()}")

    print("="*60)
    
    # Create datasets
    train_dataset = Nutrition5KDataset(
        csv_path=train_csv,
        data_root=DATA_ROOT,
        split='train',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    val_dataset = Nutrition5KDataset(
        csv_path=val_csv,
        data_root=DATA_ROOT,
        split='val',
        augment=False,
        img_size=IMG_SIZE,
    )
    
    # Create data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False,
        drop_last=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    # Use Volume  
    model = build_nutrition5k_model(
        fusion=fusion_type,
        pretrained=False,
        dropout_rate=DROPOUT_RATE,
        fusion_channels=FUSION_CHANNELS,
        use_volume=True,
    )
    model = model.to(device)
    
    print(f"Model parameters: {model.get_num_parameters():,}")
    print(f"Training samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    
    # Loss function
    criterion = nn.MSELoss()
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )
    
    print(f"Learning rate: {LEARNING_RATE}")
    print(f"Weight decay: {WEIGHT_DECAY}")
    
    # Learning rate scheduler
    steps_per_epoch = len(train_loader)
    total_steps = NUM_EPOCHS * steps_per_epoch
    warmup_steps = int(total_steps * WARMUP_RATIO)
    
    scheduler = get_warmup_cosine_scheduler(
        optimizer, 
        warmup_steps=warmup_steps, 
        total_steps=total_steps,
        min_lr_ratio=MIN_LR_RATIO
    )
    
    # Create experiment directory
    exp_name = f"inceptionv3_{fusion_type}_volume_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    exp_dir = os.path.join(OUTPUT_DIR, 'nutrition5k_experiments', exp_name)
    os.makedirs(exp_dir, exist_ok=True)
    
    # Save experiment configuration
    config = {
        'fusion': fusion_type,
        'use_volume': True,
        'pretrained': False,
        'dropout_rate': DROPOUT_RATE,
        'fusion_channels': FUSION_CHANNELS,
        'learning_rate': LEARNING_RATE,
        'weight_decay': WEIGHT_DECAY,
        'batch_size': BATCH_SIZE,
        'img_size': IMG_SIZE,
        'num_epochs': NUM_EPOCHS
    }
    
    with open(os.path.join(exp_dir, 'config.json'), 'w') as f:
        json.dump(config, f, indent=4)
    
    # Create trainer
    trainer = Trainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        device=device,
        output_dir=exp_dir,
        early_stopping_patience=EARLY_STOPPING_PATIENCE,
        scheduler_step_on_batch=False
    )
    
    # Train the model
    trainer.train(NUM_EPOCHS)
    
    print(f"\nExperiment completed! Results saved to: {exp_dir}")
    return trainer.best_metrics


volume_with_middle_fusion_result = train_nutrition5k_with_volume(fusion_type='middle')

TRAINING: Nutrition5k InceptionV3 + MIDDLE
Loaded 2804 valid samples out of 2805
Loaded 495 valid samples out of 495
Model parameters: 52,094,465
Training samples: 2804
Validation samples: 495
Learning rate: 0.0005
Weight decay: 1e-06
Starting training for 40 epochs...

Epoch 1/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.57it/s]


Train Loss: 98966.1618
Val Loss: 107401.5198
MAE: 240.62

Epoch 2/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.52it/s]


Train Loss: 69272.0114
Val Loss: 24299.0708
MAE: 107.20

Epoch 3/40


...

Train Loss: 5001.6133
Val Loss: 8364.4256
MAE: 57.44

Epoch 39/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.70it/s]


Train Loss: 5878.9752
Val Loss: 8378.5119
MAE: 59.38

Epoch 40/40


Validation: 100%|██████████| 16/16 [00:02<00:00,  6.64it/s]

Train Loss: 5337.3297
Val Loss: 9411.3538
MAE: 60.59

Training completed!
Best validation loss: 7800.9672

Experiment completed! Results saved to: ./experiments/nutrition5k_experiments/inceptionv3_middle_volume_20251025_171018


# Test Inference

In [11]:
class TestDataset(Dataset):
    """Dataset class for test set inference"""
    
    def __init__(self, test_root, img_size=256):
        self.test_root = test_root
        self.img_size = img_size
        
        # Paths to subdirectories
        self.color_dir = os.path.join(test_root, 'color')
        self.depth_raw_dir = os.path.join(test_root, 'depth_raw')
        
        # Get all dish IDs from color directory
        self.dish_ids = sorted([d for d in os.listdir(self.color_dir) 
                              if os.path.isdir(os.path.join(self.color_dir, d))])
        
        print(f"Found {len(self.dish_ids)} test samples")
        
        # Color normalization (same as training)
        self.color_normalize = T.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    
    def __len__(self):
        return len(self.dish_ids)
    
    def _resize_and_center_crop(self, img, target_size=256):
        """Resize and center crop (same as training)"""
        width, height = img.size
        
        if width < height:
            new_width = target_size
            new_height = int(target_size * height / width)
        else:
            new_height = target_size
            new_width = int(target_size * width / height)
        
        img = img.resize((new_width, new_height), Image.LANCZOS)
        
        left = (new_width - target_size) // 2
        top = (new_height - target_size) // 2
        right = left + target_size
        bottom = top + target_size
        
        return img.crop((left, top, right, bottom))
    
    def __getitem__(self, idx):
        dish_id = self.dish_ids[idx]
        
        # Load images
        rgb_path = os.path.join(self.color_dir, dish_id, 'rgb.png')
        depth_path = os.path.join(self.depth_raw_dir, dish_id, 'depth_raw.png')
        
        rgb_img = Image.open(rgb_path).convert('RGB')
        depth_img = Image.open(depth_path).convert('L')
        
        # Resize and center crop
        rgb_img = self._resize_and_center_crop(rgb_img, target_size=self.img_size)
        depth_img = self._resize_and_center_crop(depth_img, target_size=self.img_size)
        
        # Convert to tensors
        rgb_tensor = TF.to_tensor(rgb_img)
        depth_tensor = TF.to_tensor(depth_img)
        
        # Normalize
        rgb_tensor = self.color_normalize(rgb_tensor)
        depth_tensor = depth_tensor / 255.0
        
        return {
            'rgb': rgb_tensor,
            'depth': depth_tensor,
            'dish_id': dish_id
        }

In [12]:
def load_model_from_checkpoint(checkpoint_path, device='cuda'):
    """Load trained model from checkpoint"""
    
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
    
    # Get the directory to look for config
    model_dir = os.path.dirname(checkpoint_path)
    config_path = os.path.join(model_dir, 'config.json')
    
    # Load config
    if os.path.exists(config_path):
        with open(config_path, 'r') as f:
            config = json.load(f)
        print(f"Loaded config: {config}")
    else:
        raise FileNotFoundError(f"Config file not found: {config_path}")
    
    # Build model with the same configuration
    model = build_nutrition5k_model(
        fusion=config.get('fusion', 'middle'),
        pretrained=False,
        dropout_rate=config.get('dropout_rate', 0.4),
        fusion_channels=config.get('fusion_channels', 2048),
        use_volume=config.get('use_volume', False)
    )
    
    # Load state dict
    if 'model_state_dict' in checkpoint:
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Loaded model from epoch {checkpoint.get('epoch', 'unknown')}")
        if 'val_loss' in checkpoint:
            print(f"Validation loss: {checkpoint['val_loss']:.4f}")
            print(f"MAE: {checkpoint.get('mae', 'N/A')}")
    else:
        model.load_state_dict(checkpoint)
    
    model = model.to(device)
    model.eval()
    
    return model

In [13]:
@torch.no_grad()
def run_inference(model, test_dataset, batch_size=32, device='cuda'):
    """Run inference on test set"""
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    predictions = {}
    
    print("Running inference...")
    for batch in tqdm(test_loader):
        rgb = batch['rgb'].to(device)
        depth = batch['depth'].to(device)
        dish_ids = batch['dish_id']
        
        # Forward pass
        calorie_pred = model(rgb, depth)
        
        # Store predictions
        calorie_pred = calorie_pred.cpu().numpy().flatten()
        for i, dish_id in enumerate(dish_ids):
            predictions[dish_id] = float(calorie_pred[i])
    
    return predictions

def create_submission(predictions, output_path='submission.csv'):
    """Create submission CSV file"""
    
    submission_data = []
    for dish_id in sorted(predictions.keys()):
        submission_data.append({
            'ID': dish_id,
            'Value': predictions[dish_id]
        })
    
    submission_df = pd.DataFrame(submission_data)
    submission_df.to_csv(output_path, index=False)
    
    print(f"Submission saved to: {output_path}")
    print(f"Total predictions: {len(submission_data)}")
    print(f"\nSample predictions:")
    print(submission_df.head(10))
    
    # Statistics
    values = submission_df['Value'].values
    print(f"\nPrediction Statistics:")
    print(f"  Min: {values.min():.2f}")
    print(f"  Max: {values.max():.2f}")
    print(f"  Mean: {values.mean():.2f}")
    print(f"  Median: {np.median(values):.2f}")
    
    return submission_df

In [14]:
# Configuration
TEST_ROOT = './Nutrition5K/Nutrition5K/test'  # Path to test data
MODEL_PATH = './experiments/nutrition5k_experiments/inceptionv3_middle_volume_20251025_163021/best_model.pth'  # Your best model
OUTPUT_PATH = './submission.csv'

# Load test dataset
test_dataset = TestDataset(
    test_root=TEST_ROOT,
    img_size=IMG_SIZE
)

# Load model
model = load_model_from_checkpoint(MODEL_PATH, device=device)

# Run inference
predictions = run_inference(
    model=model,
    test_dataset=test_dataset,
    batch_size=BATCH_SIZE,
    device=device
)

# Create submission
submission_df = create_submission(predictions, output_path=OUTPUT_PATH)

Found 189 test samples
Loaded config: {'fusion': 'middle', 'use_volume': True, 'pretrained': False, 'dropout_rate': 0.4, 'fusion_channels': 2048, 'learning_rate': 0.0005, 'weight_decay': 1e-06, 'batch_size': 32, 'img_size': 256, 'num_epochs': 40}
Loaded model from epoch 17
Validation loss: 6987.1615
MAE: 54.757686614990234
Running inference...


100%|██████████| 6/6 [00:01<00:00,  4.13it/s]

Submission saved to: ./submission.csv
Total predictions: 189

Sample predictions:
          ID       Value
0  dish_3301  948.794556
1  dish_3302   16.694901
2  dish_3303   27.170441
3  dish_3304  222.738480
4  dish_3305  466.522308
5  dish_3306    3.652470
6  dish_3307  472.011383
7  dish_3308   44.091991
8  dish_3309  544.475403
9  dish_3310  253.851929

Prediction Statistics:
  Min: 2.25
  Max: 948.79
  Mean: 248.84
  Median: 206.76
